Data Exploration and Cleanup

Using the Bureau of Labor Statistics API to create csv files for the US weekly wages of various age groups over a 15 year time span. The wages are unadjusted for inflation and they are from the BLS's Current Population Survey.

In [3]:

#BLS is the Bureau of Labor Statistics' wrapper for their API.
import bls
import pandas as pd
import numpy as np
from config import API_Key

bls.api.set_api_key(API_Key)

#years is used as a column in the data frames later in the script
years = ['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']


#age_buckets is a list of lists containing the the Burea of Labor Statistics' ID numbers for various wage datasets\
# related to age and gender. In each element of age_buckets is a list of data set IDs and each of these IDs refer\
# to the overall, male, and female wage datasets for each age group.
age_buckets = [['LEU0252888500','LEU0252888700','LEU0252888900','25-34.csv'],['LEU0252889100','LEU0252889300','LEU0252889500','35-44.csv'], \
               ['LEU0252889700','LEU0252889900','LEU0252890100','45-54.csv'],['LEU0252890900', 'LEU0252891100', 'LEU0252891300','55-64.csv'],\
               ['LEU0252891500','LEU0252891700','LEU0252891900','65&Over.csv']]

#loop through age_buckets and create a separate csv for each age bucket
for age_bucket in age_buckets:
    
    #request a certain data set from BLS and specificy the years of data

    #first will be assigned the median US wages for the entire age bucket over the years
    first = (bls.get_series(
            age_bucket[0], startyear=2001, endyear=2015
        ))

    #second will be assigned the median US wages for males in the age bucket over the years
    second = (bls.get_series(
            age_bucket[1], startyear=2001, endyear=2015
        ))

    #third will be assigned the median US wages for females in the age bucket over the years
    third = (bls.get_series(
            age_bucket[2], startyear=2001, endyear=2015
        ))


    #first, second, and third are combined into a dataframe
    DF = pd.DataFrame({'general':first, "men": second, 'women':third})


    #The datasets returned by the BLS website return wages for Q1, Q2, Q3, and Q4 of the year \
    #Here we average the median wages for each quarter in each year and create a new DF that has wages by year

    #This collects the annual median wages in this age bucket
    general = []
    #This collects the annual median wages for men in this age bucket
    men = []
    #This collects the annual median wages for women in this age bucket
    women = []

    gen = 0
    m = 0
    w = 0
    c = 1


    #loop through each row in DF
    for i in range(len(DF['general'])):

        #add the quarterly wages from the different rows to find the annual wage
        gen +=DF['general'][i]
        m +=DF['men'][i]
        w +=DF['women'][i]

        #every fourth row execute the below code
        if c % 4 == 0:   
            #Take gen, which is the sum of the median wages for each quarter in the year, \
            # and divide by 4 to find the average of the median wages and then append that \
            # to general. The same thing is repeated for men and women.
            general.append(gen/4)
            men.append(m/4)
            women.append(w/4)

            #reset variables for next year of wages
            gen = 0
            m = 0
            w = 0

        c += 1

    #create new DF that shows annual median wages for the age bucket
    DF = pd.DataFrame({'years':years, 'general':general, "men": men, 'women':women})

    DF.to_csv('Age_Bucket_Data/{}'.format(age_bucket[3]))
    print(age_bucket[3])
    print(DF)

25-34.csv
    general     men   women years
0    576.00  617.50  512.00  2001
1    590.50  628.00  530.00  2002
2    594.25  629.00  546.25  2003
3    603.75  638.75  561.25  2004
4    610.00  644.00  572.00  2005
5    622.00  662.00  583.75  2006
6    643.00  686.75  597.50  2007
7    665.25  704.00  624.50  2008
8    678.25  715.00  634.25  2009
9    682.25  713.50  648.75  2010
10   691.75  716.75  661.75  2011
11   706.50  737.50  666.75  2012
12   708.25  743.00  665.25  2013
13   725.25  755.00  679.25  2014
14   734.75  771.25  689.50  2015
35-44.csv
    general     men   women years
0    657.00  754.00  546.50  2001
1    668.50  759.50  569.75  2002
2    687.25  775.25  590.25  2003
3    712.75  803.50  608.00  2004
4    731.00  823.25  621.00  2005
5    747.50  836.00  645.25  2006
6    769.50  872.75  667.25  2007
7    804.00  914.50  681.75  2008
8    817.75  916.00  709.00  2009
9    823.25  914.50  730.75  2010
10   837.25  935.50  733.75  2011
11   858.00  956.50  746.75 

The below code is used to merge several dataframes taken from the BLS. The BLS dataframes show weeking earnings in the US by race and by gender. Each dataframe is for one year of data.

In [4]:
#This list is used to loop through all of the race/wage CSVs
years = ['2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017']

#Read in each CSV and merge it on the "Characteric' column
Race_DF = pd.read_csv('../Raw_Data/Race_Age_Data/2002_Race_Age.csv')
print(Race_DF.head())

for year in years:
    Race_DF = pd.merge(Race_DF,pd.read_csv('../Raw_Data/Race_Age_Data/{}_Race_Age.csv'.format(year)), on = 'Characteristic')

print(Race_DF.head())

#Final dataframe
Race_DF.to_csv('Race_DataFrame.csv')

             Characteristic  2002_median_weekly_earnings 2002_workers_in_000's
0  Total, 16 years and over                          608               100,081
1    Men, 16 years and over                          679                56,345
2       Men, 16 to 24 years                          391                 6,317
3    Men, 25 years and over                          732                50,027
4  Women, 16 years and over                          529                43,737
             Characteristic  2002_median_weekly_earnings  \
0  Total, 16 years and over                          608   
1    Men, 16 years and over                          679   
2       Men, 16 to 24 years                          391   
3    Men, 25 years and over                          732   
4  Women, 16 years and over                          529   

  2002_workers_in_000's  2003_median_weekly_earnings 2003_workers_in_000's  \
0               100,081                          620               100,302   
1        

The below code cleans data about wages in certain US industries and data about the number of employees
in those industries. All of the csv files that are read in for this data cleaning process are downloaded from the 
BLS's website. The Current Population Survey is the name of the survey that collected the data.

In [5]:
#Storing file path locations

#The Age/Profession csv files contain breakdowns of the size of age groups(e.g. 25-34, 35-44)\
#for a variety of industries in the US.
age_2017 = "../Raw_Data/Gender_Age_Data/Age/Profession_Age_2017.xlsx"
age_2016 = "../Raw_Data/Gender_Age_Data/Age/Profession_Age_2016.xlsx"
age_2015 = "../Raw_Data/Gender_Age_Data/Age/Profession_Age_2015.xlsx"
age_2014 = "../Raw_Data/Gender_Age_Data/Age/Profession_Age_2014.xlsx"
age_2013 = "../Raw_Data/Gender_Age_Data/Age/Profession_Age_2013.xlsx"
age_2012 = "../Raw_Data/Gender_Age_Data/Age/Profession_Age_2012.xlsx"
age_2011 = "../Raw_Data/Gender_Age_Data/Age/Profession_Age_2011.xlsx"
#The Gender/Profession csv files contain data on the median wages for a variety of industries in\
#the US. Data for female wages in an industry and male wages in an industry is also given.
gender_2017 = "../Raw_Data/Gender_Age_Data/Gender/Profession_Gender_2017.xlsx"
gender_2016 = "../Raw_Data/Gender_Age_Data/Gender/Profession_Gender_2016.xlsx"
gender_2015 = "../Raw_Data/Gender_Age_Data/Gender/Profession_Gender_2015.xlsx"
gender_2014 = "../Raw_Data/Gender_Age_Data/Gender/Profession_Gender_2014.xlsx"
gender_2013 = "../Raw_Data/Gender_Age_Data/Gender/Profession_Gender_2013.xlsx"
gender_2012 = "../Raw_Data/Gender_Age_Data/Gender/Profession_Gender_2012.xlsx"
gender_2011 = "../Raw_Data/Gender_Age_Data/Gender/Profession_Gender_2011.xlsx"


#Creating Age DataFrames
Age_2017_df = pd.read_excel(age_2017)
Age_2016_df = pd.read_excel(age_2016)
Age_2015_df = pd.read_excel(age_2015)
Age_2014_df = pd.read_excel(age_2014)
Age_2013_df = pd.read_excel(age_2013)
Age_2012_df = pd.read_excel(age_2012)
Age_2011_df = pd.read_excel(age_2011)
#Creating Gender DataFrames
Gender_2017_df = pd.read_excel(gender_2017)
Gender_2016_df = pd.read_excel(gender_2016)
Gender_2015_df = pd.read_excel(gender_2015)
Gender_2014_df = pd.read_excel(gender_2014)
Gender_2013_df = pd.read_excel(gender_2013)
Gender_2012_df = pd.read_excel(gender_2012)
Gender_2011_df = pd.read_excel(gender_2011)

#test show header
#Age_2016_df.head()
Gender_2016_df.head()

#Merging Age and Gender DataFrames on Occupation
Profession_2017_df = pd.merge(Age_2017_df, Gender_2017_df, on="Occupation")
Profession_2016_df = pd.merge(Age_2016_df, Gender_2016_df, on="Occupation")
Profession_2015_df = pd.merge(Age_2015_df, Gender_2015_df, on="Occupation")
Profession_2014_df = pd.merge(Age_2014_df, Gender_2014_df, on="Occupation")
Profession_2013_df = pd.merge(Age_2013_df, Gender_2013_df, on="Occupation")
Profession_2012_df = pd.merge(Age_2012_df, Gender_2012_df, on="Occupation")
Profession_2011_df = pd.merge(Age_2011_df, Gender_2011_df, on="Occupation")


#Testing DataFrame Merge
Profession_2017_df.head()


#DataMunging

#Since there are hundreds of occupations listed in the above dataframe,\
#we decided to focus on a few industries that we are interested in. Industries\
#that we wanted to explore were given a value in the Industry_x column and all of\
#the other cells are left blank.
#This code removes all the rows that have a blank in the Industry_x column, \
#which leaves only the rows that we have marked to include in the final analysis.
Prof_2017 = Profession_2017_df[Profession_2017_df.Industry_x.str.contains("NaN")==False]
Prof_2016 = Profession_2016_df[Profession_2016_df.Industry_x.str.contains("NaN")==False]
Prof_2015 = Profession_2015_df[Profession_2015_df.Industry_x.str.contains("NaN")==False]
Prof_2014 = Profession_2014_df[Profession_2014_df.Industry_x.str.contains("NaN")==False]
Prof_2013 = Profession_2013_df[Profession_2013_df.Industry_x.str.contains("NaN")==False]
Prof_2012 = Profession_2012_df[Profession_2012_df.Industry_x.str.contains("NaN")==False]
Prof_2011 = Profession_2011_df[Profession_2011_df.Industry_x.str.contains("NaN")==False]

#Final Data Munging
Clean_Prof_2017 = Prof_2017.replace({'–':'0','-':'0'}, regex=True)
Clean_Prof_2016 = Prof_2016.replace({'–':'0','-':'0'}, regex=True)
Clean_Prof_2015 = Prof_2015.replace({'–':'0','-':'0'}, regex=True)
Clean_Prof_2014 = Prof_2014.replace({'–':'0','-':'0'}, regex=True)
Clean_Prof_2013 = Prof_2013.replace({'–':'0','-':'0'}, regex=True)
Clean_Prof_2012 = Prof_2012.replace({'–':'0','-':'0'}, regex=True)
Clean_Prof_2011 = Prof_2011.replace({'–':'0','-':'0'}, regex=True)


#Exporting Dataframe to CSV
Clean_Prof_2017.to_csv("Profession_Age_Gender_2017.csv", encoding = 'utf-8', index = False)
Clean_Prof_2016.to_csv("Profession_Age_Gender_2016.csv", encoding = 'utf-8', index = False)
Clean_Prof_2015.to_csv("Profession_Age_Gender_2015.csv", encoding = 'utf-8', index = False)
Clean_Prof_2014.to_csv("Profession_Age_Gender_2014.csv", encoding = 'utf-8', index = False)
Clean_Prof_2013.to_csv("Profession_Age_Gender_2013.csv", encoding = 'utf-8', index = False)
Clean_Prof_2012.to_csv("Profession_Age_Gender_2012.csv", encoding = 'utf-8', index = False)
Clean_Prof_2011.to_csv("Profession_Age_Gender_2011.csv", encoding = 'utf-8', index = False)


Below is the code for reading in and cleaning NBA wage data.
The clean data is returned in the file "NBA_clean.csv".

In [6]:
import os
import csv
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

#====================================================================================================
#=======================Building Dataset for yearly salaries with Player name as Key=================
#====================================================================================================

historic_bball = '../nbaResources/nba_salaries_1990_to_2018.csv'

# Set empty list variables
Player= []
Salary = []
Season = []

#Open current NBA CSV to zip important values into a df
with open(historic_bball, 'r') as csvFile:                    

   csvReader = csv.reader(csvFile, delimiter=',')

   #Skip headers
   next(csvReader, None)                                      

   for row in csvReader:

       #Append data from the row
       Player.append(row[0])
       Salary.append(row[1])
       Season.append(row[2])
    
#Zip lists together
cleanCSV = list(zip(Player, Salary, Season))
#cleanCSV                        #Returns raw data for processing into dataframe             


df = pd.DataFrame(cleanCSV)              #creating dataframe for\
df.columns=['Player','Salary','Season']  #changing column name
df.head()
#df.count

#====================================================================================================
#==================Building Dataset for Current Players Ages with Player name as Key=================
#====================================================================================================

bball_age = '../nbaResources/Age_player.csv'

#Set empty list variables
Player= []
Age2018 = []

#Open current NBA CSV to zip important Values into a df
with open(bball_age, 'r') as csvFile:
   csvReader = csv.reader(csvFile, delimiter=',')

   #Skip headers
   next(csvReader, None)

   for row in csvReader:

       #Append data from the row
       Player.append(row[0])
       Age2018.append(row[1])

       
#Zip lists together
AgeCSV = list(zip(Player, Age2018))
#AgeCSV

playerAge_df = pd.DataFrame(AgeCSV)
playerAge_df.columns=['Player','Age']  #changing column name
#playerAge_df.describe
playerAge_df.head()                    #Returns AGE vs Player name DF      

drop_df= df.drop_duplicates(['Player', 'Season'])    # Needed to drop players with duplicate salaries for the same season
df=drop_df



#====================================================================================================
#====================Pivot Table used to view dataset================================================
#=====================Not Needed, but I wanted to look at the dataset================================
#====================================================================================================

pivot_df=df.pivot(index="Player", columns="Season", values="Salary")

#NOT NEEDED, BUT WANTED
#=====================================================================================================
unique = df["Season"].unique()                         # Just showing the number years in my raw data
unique
#=====================================================================================================                   
Dwight= df.loc[df["Player"] == "Dwight Howard"]         # I want to check the salary of a player over all the years to verify that my merged dataframes are putting data in the correct location
Jarrett= df.loc[df["Player"] == "Jarrett Jack"]         # I want to check the salary of a player over all the years to verify that my merged dataframes are putting data in the correct location


#===========================================================================================================
#============================The Next few cells created the dataframes for the yearly salaries==============
#===========================================================================================================

df2018= df.loc[df["Season"] == "2018"]
df2018= df2018.iloc[:,[0,1]]
df2017= df.loc[df["Season"] == "2017"]
df2017= df2017.iloc[:,[0,1]]
df2016= df.loc[df["Season"] == "2016"]
df2016= df2016.iloc[:,[0,1]]
df2015= df.loc[df["Season"] == "2015"]
df2015= df2015.iloc[:,[0,1]]
#===========================2014===========================================
#===========================================================================
df2014= df.loc[df["Season"] == "2014"]
df2014= df2014.iloc[:,[0,1]]
#df2014.head()
#===========================2013===========================================
#===========================================================================
df2013= df.loc[df["Season"] == "2013"]
df2013= df2013.iloc[:,[0,1]]
#df2013.head()
#===========================2012===========================================
#===========================================================================
df2012= df.loc[df["Season"] == "2012"]
df2012= df2012.iloc[:,[0,1]]
#df2012.head()
#===========================2011===========================================
#===========================================================================
df2011= df.loc[df["Season"] == "2011"]
df2011= df2011.iloc[:,[0,1]]
#df2011.head()
#===========================2010===========================================
#===========================================================================
df2010= df.loc[df["Season"] == "2010"]
df2010= df2010.iloc[:,[0,1]]
#df2010.head()
#===========================2009===========================================
#===========================================================================
df2009= df.loc[df["Season"] == "2009"]
df2009= df2009.iloc[:,[0,1]]
#df2009.head()
#===========================2008===========================================
#===========================================================================
df2008= df.loc[df["Season"] == "2008"]
df2008= df2008.iloc[:,[0,1]]
#df2008.head()
#===========================2007===========================================
#===========================================================================
df2007= df.loc[df["Season"] == "2007"]
df2007= df2007.iloc[:,[0,1]]
#df2007.head()
#===========================2006===========================================
#===========================================================================
df2006= df.loc[df["Season"] == "2006"]
df2006= df2006.iloc[:,[0,1]]
#df2006.head()
#===========================2005===========================================#
#===========================================================================
df2005= df.loc[df["Season"] == "2005"]
df2005= df2005.iloc[:,[0,1]]
#df2005.head()
#===========================2004===========================================
#===========================================================================
df2004= df.loc[df["Season"] == "2004"]
df2004= df2004.iloc[:,[0,1]]
#df2004.head()
#===========================2003===========================================
#===========================================================================
df2003= df.loc[df["Season"] == "2003"]
df2003= df2003.iloc[:,[0,1]]
#df2003.head()
#===========================2002===========================================
#===========================================================================
df2002= df.loc[df["Season"] == "2002"]
df2002= df2002.iloc[:,[0,1]]
#df2002.head()
#===========================2001===========================================
#===========================================================================
df2001= df.loc[df["Season"] == "2001"]
df2001= df2001.iloc[:,[0,1]]
#df2001.head()
#===========================2000===========================================
#===========================================================================
df2000= df.loc[df["Season"] == "2000"]
df2000= df2000.iloc[:,[0,1]]
#df2000.head()




#=============================MERGING OF THE DATASETS JUST CREATED FOR SALARY VS YEARS===============
#====================================================================================================

Compiled_df = df2018.merge(df2017, how='left', left_on="Player", right_on='Player',suffixes=('_2018','_2017')).fillna(0)   #Left Merge Command. Left merges does not delete data
Compiled_df = Compiled_df.merge(df2016, how='left', left_on="Player", right_on='Player',suffixes=('','_2016')).fillna(0)   #Left Merge Command. Left merges does not delete data
Compiled_df = Compiled_df.merge(df2015, how='left', left_on="Player", right_on='Player',suffixes=('','_2015')).fillna(0)
Compiled_df = Compiled_df.merge(df2014, how='left', left_on="Player", right_on='Player',suffixes=('','_2014')).fillna(0)
Compiled_df = Compiled_df.merge(df2013, how='left', left_on="Player", right_on='Player',suffixes=('','_2013')).fillna(0)
Compiled_df = Compiled_df.merge(df2012, how='left', left_on="Player", right_on='Player',suffixes=('','_2012')).fillna(0)
Compiled_df = Compiled_df.merge(df2011, how='left', left_on="Player", right_on='Player',suffixes=('','_2011')).fillna(0)
Compiled_df = Compiled_df.merge(df2010, how='left', left_on="Player", right_on='Player',suffixes=('','_2010')).fillna(0)
Compiled_df = Compiled_df.merge(df2009, how='left', left_on="Player", right_on='Player',suffixes=('','_2009')).fillna(0)
Compiled_df = Compiled_df.merge(df2008, how='left', left_on="Player", right_on='Player',suffixes=('','_2008')).fillna(0)
Compiled_df = Compiled_df.merge(df2007, how='left', left_on="Player", right_on='Player',suffixes=('','_2007')).fillna(0)
Compiled_df = Compiled_df.merge(df2006, how='left', left_on="Player", right_on='Player',suffixes=('','_2006')).fillna(0)
Compiled_df = Compiled_df.merge(df2005, how='left', left_on="Player", right_on='Player',suffixes=('','_2005')).fillna(0)
Compiled_df = Compiled_df.merge(df2004, how='left', left_on="Player", right_on='Player',suffixes=('','_2004')).fillna(0)
Compiled_df = Compiled_df.merge(df2003, how='left', left_on="Player", right_on='Player',suffixes=('','_2003')).fillna(0)
Compiled_df = Compiled_df.merge(df2002, how='left', left_on="Player", right_on='Player',suffixes=('','_2002')).fillna(0)
Compiled_df = Compiled_df.merge(df2001, how='left', left_on="Player", right_on='Player',suffixes=('','_2001')).fillna(0)
Compiled_df = Compiled_df.merge(df2000, how='left', left_on="Player", right_on='Player',suffixes=('','_2000')).fillna(0)


new_col=['Player','2018', '2017', '2016', '2015', '2014', '2013', '2012',
        '2011', '2010', '2009', '2008', '2007', '2006', '2005', 
        '2004', '2003', '2002', '2001', '2000']
                                 
Compiled_df.columns = new_col
#Compiled_df


#====================================================================================================
#==================Creating final Dataframe with a Merge Function====================================
#=====Function conveniently deletes rows with index values that are not shared by both dataframes====
#====================================================================================================


Compiled_df = pd.merge(playerAge_df, Compiled_df, on="Player")
drop2_df= Compiled_df.drop_duplicates(['Player', 'Age']) # Needed to drop players with duplicate salaries for the same season
Compiled_df=drop2_df                                      #Moving final dataframe into global function

# Push the remade DataFrame to a new CSV file
Compiled_df.to_csv("NBA_clean.csv", encoding="utf-8", index=False, header=True)

print("NBA Data Cleaning ran successfully")
#END===============:)

NBA Data Cleaning ran successfully
